## TP Allocation de fréquences

Le but est d'allouer des fréquences à des antennes de telle sorte que celles qui doivent pouvoir communiquer aient des fréquences proches, spécifié par un écart maximum entre les fréquences.

- Modèle : boîte noire dans le cadre du TP.
- Paramètres : une fréquence par antenne dans un domaine qui lui est propre.
- Objectif : minimiser le nombre d'écarts non respectés.

### Quelques données sur le problème en entrée
- 50 antennes
- 4 domaines différents
- 78 écarts à respecter (donc objectif entre 0 et 78).

<img src="optimizers/donnees.png" style="width: 700px;"/>

### Etude :
- Etudier l'impact du choix de la fonction d'évaluation (voir plus bas) et de la méthode employée pour la recherche d'une solution optimale.
- Constater l'impact du choix du point de départ (aléatoire ici) sur les performances.


### Bloc de chargement du problème.

Remplacer "if False" par "if True" pour afficher les graphiques de visualisation des données.

In [2]:
%matplotlib 

import sys
sys.path.append('..')
from src.problem import load_problem
from src.evaluation import creer_matrice_ecarts
import numpy as np 
import matplotlib.pyplot as plt

# Chargement du fichier problème
problem = load_problem("../tests/optimizers/fapp05_0050.coo")


# Exploration des données :
if False: # mettre à True pour afficher la visualisation
    fig, ax = plt.subplots(2,2)
    ax[0][0].plot([0,0], [2180,2716])
    ax[0][0].plot([1,1], [2488,2584])
    ax[0][0].plot([2,2], [2384,2716])
    ax[0][0].plot([3,3], [2412,2600])
    ax[0][0].set_xlabel('Domaine')
    ax[0][0].set_ylabel('Fréquence')

    file = open('../tests/fapp05_0050.coo', 'r')
    data = dict()
    data['frequence'] = []
    data['contrainte'] = []
    data['ValContrainte'] = []
    for line in file.readlines():
        a = line.split()
        if a[0] == "TR":
            data['frequence'].append(int(a[2]))
        if a[0] == "CE":
            data['contrainte'].append(int(a[1]))
            data['contrainte'].append(int(a[2]))
            data['ValContrainte'].append(int(a[3]))
    
    print(data['frequence'])
    freq_label, freq_count = np.unique(data['frequence'], return_counts = True)
    print(freq_label, freq_count)
    ax[1][0].bar(freq_label, freq_count, align = "center")
    ax[1][0].set_xlabel('Domaine')
    ax[1][0].set_ylabel('Nombre de fréquences')
    
    contrainte_label, contrainte_count = np.unique(data['contrainte'], return_counts = True)
    ax[0][1].bar(contrainte_label, contrainte_count)
    ax[0][1].set_xlabel('Indice antenne')
    ax[0][1].set_ylabel("Nombre d'écarts")
    
    valeur_label, valeur_count = np.unique(data['ValContrainte'], return_counts = True)
    ax[1][1].bar(valeur_label, valeur_count)
    ax[1][1].set_xlabel("Valeur de l'écart")
    ax[1][1].set_ylabel("Nombre d'écarts")

    fig.tight_layout()

Using matplotlib backend: TkAgg
[2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0]
[0 1 2 3] [40  5  3  2]


### Fonction évaluation

Les différentes méthodes cherchent à optimiser un objectif, mais des fois utiliser directement cet objectif pour guider l'exploration peut faire tomber les méthodes dans des optimum locaux. Ici on utilise une fonction d'évaluation qui donne un score à chaque solution, pas forcément l'objectif lui-même, et les méthodes vont chercher à optimiser cette fonction d'évaluation. Il faut bien sûr que la minimisation de la fonction d'évaluation entraîne la minimisation de l'objectif pour que cette démarche fonctionne. Voici les fonctions d'évaluation que vous pouvez utiliser (remplacer la valeur à la ligne "evaluation = ...") :

- np.count_nonzero(eval_matrix) : nombre d'écarts non respectés (vrai objectif);
- eval_matrix.sum() : somme des différences aux écarts non respectés;
- eval_matrix.max() : maximum des différences aux écarts non respectés;
- Une combinaison des trois ou autre.

In [3]:
def fonction_evaluation(problem, solution):
    eval_matrix = creer_matrice_ecarts(problem, solution)
    evaluation = np.count_nonzero(eval_matrix)
    
    return evaluation

### Random walk

Explore aléatoirement les voisins et renvoie la meilleure solution trouvée. Paramètres modifiables :

- max_iterations : nombre d'itérations de l'algorithme.

In [4]:
from src.optimizers.random_walk import random_walk

value, solution = random_walk(problem, fonction_evaluation, max_iterations = 50)

### Random descent

Sélectionne un voisin aléatoire à chaque itération et le garde seulement si il est meilleur. Paramètres modifiables :

- max_iterations : nombre d'itérations de l'algorithme.

In [4]:
from src.optimizers.random_descent import random_descent

value, solution = random_descent(problem, fonction_evaluation, max_iterations = 50)

### Steepest descent

Sélectionne le meilleur voisin à chaque itération jusqu'à que ce ne soit plus possible. Paramètres modifiables :

- max_iterations : nombre d'itérations maximal de l'algorithme (s'arrête avant si tous les voisins sont moins bons).

In [5]:
from src.optimizers.steepest_descent import steepest_descent

value, solution =  steepest_descent(problem, fonction_evaluation, max_iterations = 50)

### Représentation tabou : tabou_element_function

A partir des éléments changés lors du choix d'une nouvelle solution, définit la représentation tabou de cette solution. Cela permettra d'exclure toutes les solutions qui ont la même représentation dans le choix du voisin. Paramètres :

- i : indice de l'antenne changée
- freq : fréquence de cette antenne
- value : valeur donnée par fonction_evaluation

Valeurs de retour possibles pour tabou_element_function :

- value : excluera les solutions pour lesquelles fonction_evaluation renvoit value. 
- i : excluera les solutions qui modifient l'antenne i.
- (i,freq) : excluera les solutions qui affectent la fréquence freq sur l'antenne i.
- (i,value) : excluera les solutions qui modifient l'antenne i et pour lesquelles fonction_evaluation renvoit value.

In [6]:
def tabou_element_function(i, freq, value):
    return value

### Méthode tabou 

A chaque itération, choisit le meilleur voisin parmi ceux qui ne sont pas exclus par les représentations tabous. Paramètres modifiables :

- max_without_improvement : nombre maximal d'itérations autorisées sans trouver de meilleure solution.
- length_list_tabou : nombre maximal de représentations tabous stockées à la fois.
- max_iterations : nombre d'itérations maximal (None pour pas de limite)

In [7]:
from src.optimizers.tabou import tabou

value, solution =  tabou(problem, fonction_evaluation, 
                                    max_without_improvement = 50, 
                                    length_list_tabou = 200,
                                    tabou_element_function = tabou_element_function,
                                    max_iterations = 200)